### Download Sentinel-2 image collection

In [ ]:
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd
import ee
import numpy as np

#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)
Map.add_basemap('SATELLITE')
Map.add_basemap('CartoDB.DarkMatter')

file_path = './data/boundaries/isimangaliso_lesser_bbox.geojson'
with open(file_path) as f:
    data = json.load(f)
    coord = data['features'][0]['geometry']['coordinates'][0][0]
    line_geom = ee.Geometry.LineString(coord)
    poly_geom = ee.Geometry.Polygon(coord)

srtm30 = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(poly_geom)
elevationVis = {'min': 3,'max': 800.0,
                'palette': [
                    '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
                    'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
                    '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f']}
Map.addLayer(srtm30, elevationVis, "SRTM 30m")

Map.addLayer(ee_object=line_geom, vis_params={'color':'red'}, name="Lesser Isimangaliso Wetland Park")
Map

In [ ]:
jrc_gsws = ee.ImageCollection("JRC/GSW1_1/YearlyHistory")
jrc_gsws_list = jrc_gsws.toList(jrc_gsws.size())
jrc_gsws_2016 = ee.Image(jrc_gsws_list.get(32)).clip(poly_geom)
vis_season = {'min':0,'max':3,'palette': ['cccccc','ffffff','99d9ea','0000ff']}
Map.addLayer(jrc_gsws_2016, vis_season, "JRC GSWS 2016")

In [ ]:
#create random points
fg_points = ee.FeatureCollection.randomPoints(poly_geom,points=300, seed=42, maxError=1)
Map.addLayer(fg_points , {'color': 'FF0000'}, 'points')

In [ ]:
collection = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(poly_geom).filterDate('2016-01-05','2016-01-31') \
    .filter(ee.Filter.inList('MGRS_TILE', ['36JVP','36JVQ'])) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .select(['B4', 'B3', 'B2'])

collection = ee.ImageCollection([ee.Image(collection_list.get(0)),ee.Image(collection_list.get(1))])
collection_list = collection.toList(collection.size())
print(f'Found: {collection.size().getInfo()} matches')

In [ ]:
for i in range(collection_list.size().getInfo()):
    data = collection_list.get(i).getInfo()['properties']['PRODUCT_ID']
    print(data)

In [ ]:
latest_meta = collection_list.get(1)
geom = ee.Image(latest_meta).geometry()
tile = latest_meta.getInfo()['properties']['PRODUCT_ID'].split('_')[5]
Map.addLayer(geom , {'color': 'FF0000'}, tile)

In [ ]:
s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(line_geom).filterDate('2019-01-01','2019-01-30') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
    .filterMetadata('resolution_meters', 'equals' , 10)

print(f'Found:{s1_collection.size().getInfo()} matches')
s1_collection_list = s1_collection.toList(s1_collection.size())


In [ ]:
for i in range(s1_collection.size().getInfo()):
    try:
        date_0 = str(pd.to_datetime(s1_collection_list.get(i).getInfo()['id'].split('_')[5].split('T')[0]).date())
        date_1 = str(pd.to_datetime(s1_collection_list.get(i+1).getInfo()['id'].split('_')[5].split('T')[0]).date())
        if date_0 == date_1:
            img_0 = ee.Image(s1_collection_list.get(i)).select('VH').clip(poly_geom)
            img_1 = ee.Image(s1_collection_list.get(i+1)).select('VH').clip(poly_geom)
            img_mos = ee.ImageCollection([img_0,img_1]).mosaic()
            Map.addLayer(img_mos,{'min': -20, 'max': 10},f'{date_1} VH')
            
    except Exception:
        pass


In [ ]:
image = collection.mosaic().clip(poly_geom)
vizParams = {
    'bands': ['B4', 'B3', 'B2'],
    'gain': [0.1, 0.1, 0.1],
    'scale':10
}

Map.addLayer(image, vizParams, '2016-01-05')


In [ ]:
geemap.ee_export_image(jrc_gsws_2016, filename='./data/test.tif', scale=20, file_per_band=False)
if os.path.exists('./data/test.tif'):
    os.remove('./data/test.zip')

In [ ]:
def get_meta_s2(meta):
    product_id = meta['properties']['PRODUCT_ID']
    date = pd.to_datetime(product_id.split('_')[2].split('T')[0]).date()
    tile = product_id.split('_')[5]
    cloud_cover = round(meta['properties']['CLOUD_COVERAGE_ASSESSMENT'],2)
    data = {'id':product_id,'date':date,'tile':tile,'cloudcover':cloud_cover}
    df = pd.DataFrame(data,index=[0])
    return df

In [ ]:
products = list(map(get_meta_s2,collection_list.getInfo()))
products_df = pd.concat(products).reset_index(drop=True)
products_df